In [9]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [10]:
migration = pd.read_csv(Path('Resources/state_to_state.csv'))
migration

,IBRC_Geo_ID,State,District Name,Year,Births,Deaths,Net International Migration,Net Domestic Migration,Residual
0,6500001,AK,Kenai Peninsula Economic Development District,1990,195,55,3.0,155,0
1,6500001,AK,Kenai Peninsula Economic Development District,1991,766,146,18.0,800,-9
2,6500001,AK,Kenai Peninsula Economic Development District,1992,736,151,43.0,129,-15
3,6500001,AK,Kenai Peninsula Economic Development District,1993,703,194,42.0,70,-12
4,6500001,AK,Kenai Peninsula Economic Development District,1994,693,192,26.0,778,-24
...,...,...,...,...,...,...,...,...,...
12842,6500418,MT,Mission West Community Development Partners,2016,493,520,9.0,367,0
12843,6500418,MT,Mission West Community Development Partners,2017,528,535,29.0,872,2
12844,6500418,MT,Mission West Community Development Partners,2018,462,512,11.0,227,-3
12845,6500418,MT,Mission West Community Development Partners,2019,428,489,27.0,589,-5


In [20]:
# Grouping by the 'State' column
grouped_by_state = migration.groupby('State')

# Now, you can perform operations on each group, for example, calculating the mean
by_state = grouped_by_state.sum()

slice_year = migration[(migration['Year'] >= 1999) & (migration['Year'] <= 2024)]
slice_year = slice_year.drop(['District Name','IBRC_Geo_ID'], axis=1)
slice_year

,State,Year,Births,Deaths,Net International Migration,Net Domestic Migration,Residual
9,AK,1999,6463,1876,274.0,3395,-103
10,AK,1999,678,246,31.0,218,-9
11,AK,2000,135,80,3.0,-99,-2
12,AK,2001,622,278,20.0,81,-57
13,AK,2002,615,293,17.0,487,-40
...,...,...,...,...,...,...,...
12842,MT,2016,493,520,9.0,367,0
12843,MT,2017,528,535,29.0,872,2
12844,MT,2018,462,512,11.0,227,-3
12845,MT,2019,428,489,27.0,589,-5


In [22]:
grouped_df = slice_year.groupby(['State', 'Year']).sum()
grouped_df = grouped_df.reset_index()
#ca_df = grouped_df[grouped_df['State'] == 'CA']

In [24]:
grouped_df['Percentage']=grouped_df['Net Domestic Migration'].pct_change()
grouped_df = grouped_df.dropna()

In [8]:
import hvplot.pandas
grouped_df.hvplot(groupby='State',
                x='Year', 
                y='Net Domestic Migration',
                xlabel='Year', 
                ylabel='Domestic Migration',
                title='Domestic Migration Change by State').opts(yformatter='%.0f',
                                      active_tools=[])

:DynamicMap   [State]
   :Curve   [Year]   (Net Domestic Migration)

In [25]:
grouped_df

,State,Year,Births,Deaths,Net International Migration,Net Domestic Migration,Residual,Percentage
1,AK,2000,455,188,61.0,-497,-9,-0.966218
2,AK,2001,1880,783,251.0,-2380,-108,3.788732
3,AK,2002,1873,736,157.0,-521,-107,-0.781092
4,AK,2003,1871,770,-53.0,-1176,-119,1.257198
5,AK,2004,1791,774,335.0,-1102,-138,-0.062925
...,...,...,...,...,...,...,...,...
1007,WV,2016,19342,22458,1290.0,-9056,-15,0.512106
1008,WV,2017,18952,23139,592.0,-10139,-18,0.119589
1009,WV,2018,18589,23625,54.0,-7734,-14,-0.237203
1010,WV,2019,17839,22767,1185.0,-6929,-18,-0.104086
